# Sistemas Inteligentes 2021/2022

## Mini-projeto 2: Quadrados Latinos


## Grupo: 42

### Elementos do Grupo

Número:  53687       Nome:   Ariana Dias  
Número:  53746       Nome:   Andrei Tataru  
Número:  51127       Nome:   Luís Ferreirinha  

(Nota: Neste relatório pode adicionar as células de texto e código que achar necessárias.)

## Representação de variáveis, domínios, vizinhos e restrições

O problema dos quadrados latinos pode ser representado como uma matriz $n \times n$ em que cada linha e coluna contêm exatmente um número de $1$ a $n$.  

Então para caracterizar este CSP em python obtamos pelas seguintes escolhas:  

- Variáveis:  
  
    Cada variável no nosso problema vai representar o valor atribuiado a um elemento da matriz.  
    Então em python representamos o conjunto das variáveis como uma lista das coordenadas de cada
    elemento na matriz:    
    -  var $=[(i_1,j_1),...,(i_n,j_n)]$  
<br>
- Domínios:  
  
    Neste problema cada variável pode tomar um valor de $1,...,n$, exceto de houver condições iniciais.
    Portanto em python decidimos utilizar um dicionário, onde as chaves são as variáveis e estas correspondem a uma lista de inteiros contento os valores possíveis para aquela variável:
    
    - dominio $=\{(i_1,j_1) = [1,..,n], ...\}$  
<br> 
- Vizinhos:  

    Uma variável vai ser vizinha de outra de estiver na mesma linha ou coluna que essa variável.
    Em python definimos os vizinhos como um dicionário onde cada variável é uma chave e corresponde a uma lista de variáveis que são suas vizinhas:  
    
    - vizinhos $=\{(i_k,i_m) : [(i_1,j_m),...,(i_n,j_m),(i_k,j_1),...,(i_k,j_n)], \, ...  \, \}$  
<br>
- Restrições:  

    A restrição principal deste problema é a impossibilidade de haver repetições de um número
    ao longo de uma coluna ou linha, ou seja tem de ser diferente de todos os seus vizinhos.
    Em python esta restrição é representada pela comparação dos valores de duas variáveis vizinhas:  

    -  ```python
       valor_variavel_1 != valor_variavel_2
       ```
    <br>
    Como também temos a variação do Futoshiki, temos de definir outras restrições para esse problema.  

    A ideia principal é que duas variáveis adjacentes podem agora ter a restrição adicional de o valor de uma ser maior ou menor que o valor da outra. Então para representar o mapeamento destas novas restrições usamos um dicionário onde as chaves são um tuplos contendos as duas variáveis em questão e vão corresponde a uma função que verifica se os valores destas são menores ou maiores conforme a restrição.

    - desigualdades $=\{((i_k,j_m),(i_k,j_{m+1})) : maior\, \, \text{ou} \, \, menor \}$  

    Cada uma destas funções vair comparar os valores das variavéis conforme o seu nome.  

    - ```python 
      def maior(a,b): return a > b
      ```

    - ```python 
      def menor(a,b): return a < b
      ```


 

## Formulação do problema

In [37]:
from csp import *

def quadrado_latino(n=3, initial={}) -> CSP:
    """
    Pode receber parametros ou não.
    Deve devolver um CSP, à semelhança dos guiões das aulas PL.
    Comente o código.
    """
    variaveis = [(i,j) for i in range(0,n) for j in range(0,n)]

    conjunto = [i for i in range(1,n+1)]
    
    dominios = initial

    # Diferenca dos dois conjuntos vai resultar nas variaveis nao preenchidas
    for item in set(variaveis - initial.keys()):
        dominios[item] = conjunto
    
    condicao_vizinhanca = lambda x,y : (x[0] == y[0] and x[1] != y[1]) or (x[0] != y[0] and x[1] == y[1])
    
    vizinhos = {var : list(filter(lambda x: condicao_vizinhanca(x,var), variaveis)) for var in variaveis}

    def restricoes(X, a, Y, b):
        if Y in vizinhos[X]:
            return a != b
        return True
        
    return CSP(variaveis, dominios, vizinhos, restricoes)

## Criação do problema do quadrado latino simples

Mostrem que o código está a funcionar, construindo um problema de quadrado latino *4x4*, imprimindo as variáveis, domínios iniciais, e vizinhos. Adicione os comentários necessários. Mostre como podemos criar um puzzle com quadrados já preenchidos, e qual o impacto que isso tem nas variáveis, domínios iniciais, e vizinhos.

In [38]:
p2 = quadrado_latino(4)
print("Variáveis = ", p2.variables)
print("Domínios = ", p2.domains)
print("Vizinhos = ", p2.neighbors)

Variáveis =  [(0, 0), (1, 0), (2, 0), (3, 0), (0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (2, 2), (3, 2), (0, 3), (1, 3), (2, 3), (3, 3)]
Domínios =  {(0, 1): [1, 2, 3, 4], (1, 2): [1, 2, 3, 4], (2, 1): [1, 2, 3, 4], (3, 1): [1, 2, 3, 4], (0, 2): [1, 2, 3, 4], (2, 2): [1, 2, 3, 4], (1, 0): [1, 2, 3, 4], (3, 2): [1, 2, 3, 4], (1, 3): [1, 2, 3, 4], (0, 0): [1, 2, 3, 4], (1, 1): [1, 2, 3, 4], (0, 3): [1, 2, 3, 4], (2, 0): [1, 2, 3, 4], (3, 0): [1, 2, 3, 4], (2, 3): [1, 2, 3, 4], (3, 3): [1, 2, 3, 4]}
Vizinhos =  {(0, 0): [(1, 0), (2, 0), (3, 0), (0, 1), (0, 2), (0, 3)], (1, 0): [(0, 0), (2, 0), (3, 0), (1, 1), (1, 2), (1, 3)], (2, 0): [(0, 0), (1, 0), (3, 0), (2, 1), (2, 2), (2, 3)], (3, 0): [(0, 0), (1, 0), (2, 0), (3, 1), (3, 2), (3, 3)], (0, 1): [(0, 0), (1, 1), (2, 1), (3, 1), (0, 2), (0, 3)], (1, 1): [(1, 0), (0, 1), (2, 1), (3, 1), (1, 2), (1, 3)], (2, 1): [(2, 0), (0, 1), (1, 1), (3, 1), (2, 2), (2, 3)], (3, 1): [(3, 0), (0, 1), (1, 1), (2, 1), (3, 2), (3, 3)], (0, 2): [(0, 0),

Para criar um puzzle com quadrados já preenchidos, alteramos a função de formulação do problema, fazendo com que esta agora receba também um dicionário com os quadrados já preenchidos.

O facto de o quadrado ser semi preenchido apenas vai afetar os dominios iniciais, fazendo com que o dominio inicial das variaveis ja com valores atribuidos seja reduzido para essas.


In [39]:
preenchido = {(0,0): [1], (1,1):[3], (2,2):[2]}

pSP = quadrado_latino(3, preenchido)
print("Variáveis = ", pSP.variables)
print("Domínios = ", pSP.domains)
print("Vizinhos = ", pSP.neighbors)

r = backtracking_search(pSP)
print(r)

Variáveis =  [(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1), (0, 2), (1, 2), (2, 2)]
Domínios =  {(0, 0): [1], (1, 1): [3], (2, 2): [2], (0, 1): [1, 2, 3], (0, 2): [1, 2, 3], (1, 2): [1, 2, 3], (2, 1): [1, 2, 3], (1, 0): [1, 2, 3], (2, 0): [1, 2, 3]}
Vizinhos =  {(0, 0): [(1, 0), (2, 0), (0, 1), (0, 2)], (1, 0): [(0, 0), (2, 0), (1, 1), (1, 2)], (2, 0): [(0, 0), (1, 0), (2, 1), (2, 2)], (0, 1): [(0, 0), (1, 1), (2, 1), (0, 2)], (1, 1): [(1, 0), (0, 1), (2, 1), (1, 2)], (2, 1): [(2, 0), (0, 1), (1, 1), (2, 2)], (0, 2): [(0, 0), (0, 1), (1, 2), (2, 2)], (1, 2): [(1, 0), (1, 1), (0, 2), (2, 2)], (2, 2): [(2, 0), (2, 1), (0, 2), (1, 2)]}
{(0, 0): 1, (1, 0): 2, (2, 0): 3, (0, 1): 2, (1, 1): 3, (2, 1): 1, (0, 2): 3, (1, 2): 1, (2, 2): 2}


Resolva o problema com o backtracking sem inferencia, com inferencia, e com uma heurística.

In [40]:
import timeit
tempos = {}
solucoes = {}
n = 3

In [41]:
QuadradoLatinoSemInferência = quadrado_latino(n)
start = timeit.default_timer()
r = backtracking_search(QuadradoLatinoSemInferência)
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Assignment (QuadradoLatinoSemInferência) = ',r)
tempos['QuadradoLatinoSemInferência'] = stop - start
solucoes['QuadradoLatinoSemInferência'] = r

Time:  0.00027939999927184545
Assignment (QuadradoLatinoSemInferência) =  {(0, 0): 1, (1, 0): 2, (2, 0): 3, (0, 1): 2, (1, 1): 1, (2, 1): 4, (0, 2): 3, (1, 2): 4, (2, 2): 1}


In [42]:
QuadradoLatinoComInferência_FRWDchecking = quadrado_latino(n)
start = timeit.default_timer()
r = backtracking_search(QuadradoLatinoComInferência_FRWDchecking, inference=forward_checking)
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Assignment (QuadradoLatinoComInferência_FRWDchecking) = ',r)
tempos['QuadradoLatinoComInferência_FRWDchecking'] = stop - start
solucoes['QuadradoLatinoComInferência_FRWDchecking'] = r

Time:  0.00030280000100901816
Assignment (QuadradoLatinoComInferência_FRWDchecking) =  {(0, 0): 1, (1, 0): 2, (2, 0): 3, (0, 1): 2, (1, 1): 1, (2, 1): 4, (0, 2): 3, (1, 2): 4, (2, 2): 1}


In [43]:
QuadradoLatinoComAC3 = quadrado_latino(n)
start = timeit.default_timer()
p_AC3 = AC3(QuadradoLatinoComAC3)
r = backtracking_search(p_AC3)
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Assignment (QuadradoLatinoComAC3) = ',r)
tempos['QuadradoLatinoComAC3'] = stop - start
solucoes['QuadradoLatinoComAC3'] = r

Time:  0.0005563999984588008
Assignment (QuadradoLatinoComAC3) =  {(0, 0): 1, (1, 0): 2, (2, 0): 3, (0, 1): 2, (1, 1): 1, (2, 1): 4, (0, 2): 3, (1, 2): 4, (2, 2): 1}


In [44]:
QuadradoLatinoComInferência_MAC = quadrado_latino(n)
start = timeit.default_timer()
r = backtracking_search(QuadradoLatinoComInferência_MAC, inference=mac)
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Assignment (QuadradoLatinoComInferência_MAC) = ',r)
tempos['QuadradoLatinoComInferência_MAC'] = stop - start
solucoes['QuadradoLatinoComInferência_MAC'] = r

Time:  0.0006183000004966743
Assignment (QuadradoLatinoComInferência_MAC) =  {(0, 0): 1, (1, 0): 2, (2, 0): 3, (0, 1): 2, (1, 1): 1, (2, 1): 4, (0, 2): 3, (1, 2): 4, (2, 2): 1}


In [45]:
QuadradoLatinoComAC3eInferência_MAC = quadrado_latino(n)
start = timeit.default_timer()
p_AC3 = AC3(QuadradoLatinoComAC3eInferência_MAC)
r = backtracking_search(p_AC3, inference=mac)
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Assignment (QuadradoLatinoComAC3eInferência_MAC) = ',r)
tempos['QuadradoLatinoComAC3eInferência_MAC'] = stop - start
solucoes['QuadradoLatinoComAC3eInferência_MAC'] = r

Time:  0.0008835999997245381
Assignment (QuadradoLatinoComAC3eInferência_MAC) =  {(0, 0): 1, (1, 0): 2, (2, 0): 3, (0, 1): 2, (1, 1): 1, (2, 1): 4, (0, 2): 3, (1, 2): 4, (2, 2): 1}


In [46]:
QuadradoLatinoComHeuristica = quadrado_latino(n)
start = timeit.default_timer()
r = backtracking_search(QuadradoLatinoComHeuristica, select_unassigned_variable = mrv)
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Assignment (QuadradoLatinoComHeuristica) = ',r)
tempos['QuadradoLatinoComHeuristica'] = stop - start
solucoes['QuadradoLatinoComHeuristica'] = r

Time:  0.00048299999980372377
Assignment (QuadradoLatinoComHeuristica) =  {(0, 2): 1, (1, 2): 2, (1, 0): 1, (2, 2): 3, (0, 1): 2, (2, 1): 1, (0, 0): 3, (1, 1): 3, (2, 0): 2}


In [47]:
QuadradoLatinoComHeuristicaComInferencia = quadrado_latino(n)
start = timeit.default_timer()
r = backtracking_search(QuadradoLatinoComHeuristicaComInferencia, select_unassigned_variable = mrv, inference=forward_checking)
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Assignment (QuadradoLatinoComHeuristicaComInferencia) = ',r)
tempos['QuadradoLatinoComHeuristicaComInferencia'] = stop - start
solucoes['QuadradoLatinoComHeuristicaComInferencia'] = r

Time:  0.0005094999996799743
Assignment (QuadradoLatinoComHeuristicaComInferencia) =  {(1, 0): 1, (1, 2): 2, (1, 1): 3, (2, 2): 1, (0, 2): 3, (2, 1): 2, (0, 1): 1, (0, 0): 2, (2, 0): 3}


In [48]:
import pandas as pd
temposSorted = dict(sorted(tempos.items(), key=lambda x:x[1]))
pd.DataFrame(temposSorted.values(),index = temposSorted.keys(), columns= [n])


,3
QuadradoLatinoSemInferência,0.000279
QuadradoLatinoComInferência_FRWDchecking,0.000303
QuadradoLatinoComHeuristica,0.000483
QuadradoLatinoComHeuristicaComInferencia,0.000509
QuadradoLatinoComAC3,0.000556
QuadradoLatinoComInferência_MAC,0.000618
QuadradoLatinoComAC3eInferência_MAC,0.000884


## Big Duvidas AQUI

Com a tabela acima é possível concluir que:
- o algoritmo que demora menos tempo a encontrar uma solução é a **procura com Retrocesso usando a inferência Forward Checking**, demora apenas 0.00046 segundos;
- a solução do problema utilizando a procura com retrocesso sem inferências nem heurísticas demora mais tempo (0.000991 segundos) que a procura com retrocesso depois de pré-processamento com recurso ao algoritmo **AC3** (0.000890 segundos);
- a procura com retrocesso usando Inferência e Heuristica (0.000997 segundos) é mais lenta que a procura sem (0.000991 segundos), sendo esta diferença na ordem dos 10 microsegundos;
- independentemente da realização de um pré-processamento com AC3, a procura com inferência em que se mantém a consistência (mac), demora mais tempo, diferença na ordem dos 10 milisegundos;

In [49]:
sols = pd.DataFrame(solucoes).T
sols

,0,1,2,0,1,2,0,1,2
,0,0,0,1,1,1,2,2,2
QuadradoLatinoSemInferência,1,2,3,2,1,4,3,4,1
QuadradoLatinoComInferência_FRWDchecking,1,2,3,2,1,4,3,4,1
QuadradoLatinoComAC3,1,2,3,2,1,4,3,4,1
QuadradoLatinoComInferência_MAC,1,2,3,2,1,4,3,4,1
QuadradoLatinoComAC3eInferência_MAC,1,2,3,2,1,4,3,4,1
QuadradoLatinoComHeuristica,3,1,2,2,3,1,1,2,3
QuadradoLatinoComHeuristicaComInferencia,2,1,3,1,3,2,3,2,1


In [50]:
sols.drop_duplicates()

,0,1,2,0,1,2,0,1,2
,0,0,0,1,1,1,2,2,2
QuadradoLatinoSemInferência,1,2,3,2,1,4,3,4,1
QuadradoLatinoComHeuristica,3,1,2,2,3,1,1,2,3
QuadradoLatinoComHeuristicaComInferencia,2,1,3,1,3,2,3,2,1


## Visualização do problema

In [51]:
def visualizacao_quadrado_latino(n, problema_csp, solucao):
    
    estado_inicial = [[0 for y in range(n)] for x in range(n)]
    for var, dom in problema_csp.domains.items():
        if len(dom) == 1:
            estado_inicial[var[1]][var[0]] = dom[0]
        
    estado_final = [[0 for y in range(n)] for x in range(n)]
    for var, valor in solucao.items():
        estado_final[var[1]][var[0]] = valor
    
    board_inicial = ""
    board_final = ""
    for y in range(n):
        for x in range(n):
            board_inicial += str(estado_inicial[y][x]) + " "
            board_final += str(estado_final[y][x]) + " "
        board_inicial += "\n"
        board_final += "\n"
    
    print("Estado Inicial:")
    print(board_inicial)
    print("Solução:")
    print(board_final)

In [52]:
visualizacao_quadrado_latino(3, quadrado_latino(3), backtracking_search(quadrado_latino(3)))

Estado Inicial:
0 0 0 
0 0 0 
0 0 0 

Solução:
1 2 3 
2 1 4 
3 4 1 



In [53]:
p3 = quadrado_latino(3)
print("Variáveis = ", p3.variables)
print("Domínios = ", p3.domains)
print("Vizinhos = ", p3.neighbors)

Variáveis =  [(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1), (0, 2), (1, 2), (2, 2)]
Domínios =  {(0, 1): [1, 2, 3, 4], (1, 2): [1, 2, 3, 4], (2, 1): [1, 2, 3, 4], (3, 1): [1, 2, 3, 4], (0, 2): [1, 2, 3, 4], (2, 2): [1, 2, 3, 4], (1, 0): [1, 2, 3, 4], (3, 2): [1, 2, 3, 4], (1, 3): [1, 2, 3, 4], (0, 0): [1, 2, 3, 4], (1, 1): [1, 2, 3, 4], (0, 3): [1, 2, 3, 4], (2, 0): [1, 2, 3, 4], (3, 0): [1, 2, 3, 4], (2, 3): [1, 2, 3, 4], (3, 3): [1, 2, 3, 4]}
Vizinhos =  {(0, 0): [(1, 0), (2, 0), (0, 1), (0, 2)], (1, 0): [(0, 0), (2, 0), (1, 1), (1, 2)], (2, 0): [(0, 0), (1, 0), (2, 1), (2, 2)], (0, 1): [(0, 0), (1, 1), (2, 1), (0, 2)], (1, 1): [(1, 0), (0, 1), (2, 1), (1, 2)], (2, 1): [(2, 0), (0, 1), (1, 1), (2, 2)], (0, 2): [(0, 0), (0, 1), (1, 2), (2, 2)], (1, 2): [(1, 0), (1, 1), (0, 2), (2, 2)], (2, 2): [(2, 0), (2, 1), (0, 2), (1, 2)]}


## Criação do problema Futoshiki *5x5*

Mostrem que o código está a funcionar, construindo um problema de Futoshiki *5x5*, imprimindo as variáveis, domínios iniciais, e vizinhos. Adicione os comentários necessários. Utilize o [link](https://www.futoshiki.org/) para gerar puzzles e validar a implementação.

In [54]:
def maior(x,y):
    return x > y

def menor(x,y):
    return x < y

def futoshiki(n=3, initial={}, desigualdades={}):
    """
    Pode receber parametros ou não.
    Deve devolver um CSP, à semelhança dos guiões das aulas PL.
    Comente o código.
    """
    variaveis = [(i,j) for j in range(0,n) for i in range(0,n)]

    conjunto = [i for i in range(1,n+1)]
    
    dominios = initial

    # Diferenca dos dois conjuntos vai resultar nas variaveis nao preenchidas
    for item in set(variaveis - initial.keys()):
        dominios[item] = conjunto
    
    condicao_vizinhanca = lambda x,y : ((x[0] == y[0] and x[1] != y[1]) or (x[0] != y[0] and x[1] == y[1]))
    
    vizinhos = {var : list(filter(lambda x: condicao_vizinhanca(x,var), variaveis)) for var in variaveis}

    def restricoes(X, a, Y, b):
        if (X,Y) in desigualdades.keys():
            if not desigualdades[(X,Y)](a,b):
                return False
        if (Y,X) in desigualdades.keys():
            if not desigualdades[(Y,X)](b,a):
                return False
        if Y in vizinhos[X]:
            return  a != b
        return True

    return CSP(variaveis, dominios, vizinhos, restricoes)

In [55]:
preenchidos = {(1,3) : [1]}
desigualdades = {((1,0),(2,0)) : maior, ((2,0),(3,0)) : maior, ((1,1),(1,0)): maior, ((3,3),(3,2)) : maior}

In [56]:
p = futoshiki(4, preenchidos, desigualdades)
print("Variáveis = ", p.variables)
print("Domínios = ", p.domains)
print("Vizinhos = ", p.neighbors)
r = backtracking_search(p)

Variáveis =  [(0, 0), (1, 0), (2, 0), (3, 0), (0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (2, 2), (3, 2), (0, 3), (1, 3), (2, 3), (3, 3)]
Domínios =  {(1, 3): [1], (0, 1): [1, 2, 3, 4], (1, 2): [1, 2, 3, 4], (2, 1): [1, 2, 3, 4], (0, 0): [1, 2, 3, 4], (3, 1): [1, 2, 3, 4], (1, 1): [1, 2, 3, 4], (0, 3): [1, 2, 3, 4], (2, 0): [1, 2, 3, 4], (3, 0): [1, 2, 3, 4], (2, 3): [1, 2, 3, 4], (0, 2): [1, 2, 3, 4], (3, 3): [1, 2, 3, 4], (2, 2): [1, 2, 3, 4], (1, 0): [1, 2, 3, 4], (3, 2): [1, 2, 3, 4]}
Vizinhos =  {(0, 0): [(1, 0), (2, 0), (3, 0), (0, 1), (0, 2), (0, 3)], (1, 0): [(0, 0), (2, 0), (3, 0), (1, 1), (1, 2), (1, 3)], (2, 0): [(0, 0), (1, 0), (3, 0), (2, 1), (2, 2), (2, 3)], (3, 0): [(0, 0), (1, 0), (2, 0), (3, 1), (3, 2), (3, 3)], (0, 1): [(0, 0), (1, 1), (2, 1), (3, 1), (0, 2), (0, 3)], (1, 1): [(1, 0), (0, 1), (2, 1), (3, 1), (1, 2), (1, 3)], (2, 1): [(2, 0), (0, 1), (1, 1), (3, 1), (2, 2), (2, 3)], (3, 1): [(3, 0), (0, 1), (1, 1), (2, 1), (3, 2), (3, 3)], (0, 2): [(0, 0), (0, 1), 

In [57]:
print(r)

{(0, 0): 4, (1, 0): 3, (2, 0): 2, (3, 0): 1, (0, 1): 3, (1, 1): 4, (2, 1): 1, (3, 1): 2, (0, 2): 1, (1, 2): 2, (2, 2): 4, (3, 2): 3, (0, 3): 2, (1, 3): 1, (2, 3): 3, (3, 3): 4}


Resolva o problema com o backtracking sem inferencia, com inferencia, e com uma heurística. Até que dimensão consegue resolver o problema em menos de 1 minuto?

#### Problema Futoshiki *5x5*

In [58]:
preenchidos5={(0,4):[2], (3,0):[4]}
desigualdades5 = { ((0,0),(0,1)) : maior, ((1,0),(2,0)) : maior,((1,1),(2,1)) : menor , ((1,3),(2,3)) : maior, ((3,1),(3,2)) : menor, ((2,3),(3,3)) : menor, ((3,1),(4,1)) : maior, ((3,2),(4,2)) : menor}
futoshiki5 = futoshiki(5, preenchidos5, desigualdades5)

In [59]:
print("Variáveis = ", futoshiki5.variables)
print("Domínios = ", futoshiki5.domains)
print("Vizinhos = ", futoshiki5.neighbors)

Variáveis =  [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (0, 3), (1, 3), (2, 3), (3, 3), (4, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4)]
Domínios =  {(0, 4): [2], (3, 0): [4], (0, 1): [1, 2, 3, 4, 5], (2, 4): [1, 2, 3, 4, 5], (4, 0): [1, 2, 3, 4, 5], (1, 2): [1, 2, 3, 4, 5], (3, 4): [1, 2, 3, 4, 5], (2, 1): [1, 2, 3, 4, 5], (4, 3): [1, 2, 3, 4, 5], (3, 1): [1, 2, 3, 4, 5], (0, 2): [1, 2, 3, 4, 5], (2, 2): [1, 2, 3, 4, 5], (1, 0): [1, 2, 3, 4, 5], (3, 2): [1, 2, 3, 4, 5], (1, 3): [1, 2, 3, 4, 5], (4, 1): [1, 2, 3, 4, 5], (4, 4): [1, 2, 3, 4, 5], (0, 0): [1, 2, 3, 4, 5], (1, 1): [1, 2, 3, 4, 5], (0, 3): [1, 2, 3, 4, 5], (2, 0): [1, 2, 3, 4, 5], (4, 2): [1, 2, 3, 4, 5], (1, 4): [1, 2, 3, 4, 5], (2, 3): [1, 2, 3, 4, 5], (3, 3): [1, 2, 3, 4, 5]}
Vizinhos =  {(0, 0): [(1, 0), (2, 0), (3, 0), (4, 0), (0, 1), (0, 2), (0, 3), (0, 4)], (1, 0): [(0, 0), (2, 0), (3, 0), (4, 0), (1, 1), (1, 2), (1, 3), (1, 4)], (2, 0): [(0

In [60]:
r = backtracking_search(futoshiki5)
print(r)

None


#### Problema Futoshiki *6x6*

In [61]:
preenchidos = {(4,3): [1], (5,4): [2]}
desigualdades = {((0,0),(1,0)):menor, ((1,0),(1,1)):menor, ((0,2),(1,2)):maior, ((3,0),(3,1)):maior, ((3,2),(3,3)):menor, ((2,4),(3,4)):maior, ((3,3),(3,4)):menor, ((3,0),(4,0)):menor, ((4,0),(4,1)):menor, ((4,1),(4,2)):menor, ((5,0),(5,1)):maior, ((5,1),(5,2)):maior}
futoshiki6 = futoshiki(6, preenchidos, desigualdades)

In [62]:
print("Variáveis = ", futoshiki6.variables)
print("Domínios = ", futoshiki6.domains)
print("Vizinhos = ", futoshiki6.neighbors)

Variáveis =  [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (0, 3), (1, 3), (2, 3), (3, 3), (4, 3), (5, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5)]
Domínios =  {(4, 3): [1], (5, 4): [2], (4, 0): [1, 2, 3, 4, 5, 6], (3, 4): [1, 2, 3, 4, 5, 6], (3, 1): [1, 2, 3, 4, 5, 6], (5, 1): [1, 2, 3, 4, 5, 6], (0, 2): [1, 2, 3, 4, 5, 6], (0, 5): [1, 2, 3, 4, 5, 6], (2, 2): [1, 2, 3, 4, 5, 6], (1, 0): [1, 2, 3, 4, 5, 6], (2, 5): [1, 2, 3, 4, 5, 6], (1, 3): [1, 2, 3, 4, 5, 6], (4, 2): [1, 2, 3, 4, 5, 6], (3, 0): [1, 2, 3, 4, 5, 6], (4, 5): [1, 2, 3, 4, 5, 6], (3, 3): [1, 2, 3, 4, 5, 6], (5, 0): [1, 2, 3, 4, 5, 6], (5, 3): [1, 2, 3, 4, 5, 6], (0, 1): [1, 2, 3, 4, 5, 6], (2, 4): [1, 2, 3, 4, 5, 6], (1, 2): [1, 2, 3, 4, 5, 6], (0, 4): [1, 2, 3, 4, 5, 6], (2, 1): [1, 2, 3, 4, 5, 6], (1, 5): [1, 2, 3, 4, 5, 6], (3, 2): [1, 2, 3, 4, 5, 6], (4, 1): [1, 2, 

In [63]:
r = backtracking_search(futoshiki6)

In [68]:
visualizacao_futoshiki(6, futoshiki6, desigualdades, r)

Estado Inicial:
0 < 0   0   0 < 0   0
                ∨   ∧
0   0   0   0   0   0
            ∨        
0 > 0   0   0   0   0
            ∨        
0   0   0   0   1   0
                     
0   0   0 > 0   0   2
                     
0   0   0   0   0   0
    ∨       ∧   ∨   ∧

Solução:
1 < 2   5   3 < 4   6
                ∨   ∧
6   3   2   1   5   4
            ∨        
5 > 4   1   2   6   3
            ∨        
2   6   3   4   1   5
                     
4   1   6 > 5   3   2
                     
3   5   4   6   2   1
    ∨       ∧   ∨   ∧



## Visualização do problema

In [65]:
# Implemente uma função que permita visualizar o puzzle Futoshiki, antes e depois de resolvido. Compare com a solução obtida pelo seu algoritmo. 
# No caso de não implementar esta função, inclua um screenshot do problema e da sua solução.
def visualizacao_futoshiki(n, problema_csp, desigualdades, solucao):
    
    def mapeamento_2D_para_1D(x,y):
        return y*(n+(n-1)*3+1) + x

    def escrita_numeros(x,y):
        return mapeamento_2D_para_1D(x*4, y*2)
    
    def escrita_desigualdades_linha(x,y):
        return mapeamento_2D_para_1D(x*4+2, y*2)
    
    def escrita_desigualdades_coluna(x,y):
        return mapeamento_2D_para_1D(x*4, y*2-1)

    def escrever_desigualdade(operacao, direcao):
        if direcao == "x":
            if operacao.__name__ == "maior":
                return '>'
            else:
                return '<'
        else:
            if operacao.__name__ == "maior":
                return u"\u2227" # Carater ∧
            else:
                return u"\u2228" # Carater ∨

    board_inicial = ([" "] * (n+(n-1)*3) + ["\n"]) * (n*2)
    for var, dom in problema_csp.domains.items():
        if len(dom) == 1:
            board_inicial[escrita_numeros(var[0],var[1])] = str(dom[0])
        else:
            board_inicial[escrita_numeros(var[0],var[1])] = "0"
    for var, operacao in desigualdades.items():
        if (var[0][1] == var[1][1]):
            board_inicial[escrita_desigualdades_linha(var[0][0],var[0][1])] = escrever_desigualdade(operacao, "x")
        else:
            board_inicial[escrita_desigualdades_coluna(var[0][0],var[0][1])] = escrever_desigualdade(operacao, "y")
    
    board_final = board_inicial.copy()
    for var, valor in solucao.items():
        board_final[escrita_numeros(var[0],var[1])] = str(valor)
    

    print("Estado Inicial:")
    print(''.join(board_inicial))
    print("Solução:")
    print(''.join(board_final))